In [2]:
import requests
from lxml import html
import pandas as pd
import json
import time
from tqdm import tqdm_notebook as tqdm

In [3]:
df = pd.read_excel('/Users/mayankmahawar/Documents/GitHub/WebScrap/Book1.xlsx', sheet_name = 'Sheet1')

In [4]:
df.head()

,Insurance Sectors,Company Name,Matched Name,Company Number,Company Number Formatted,Company Number.1,Match % Standard,Match %,Match Score,"GWP, Revenue",...,Expense ratio,# of Employees,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode
0,Commercial and Personal,Ageas,AGEAS (UK) LIMITED,354568,1093301,01093301,0.319395,0.372547,0.372547,€14.01 billion (end 2013)[1],...,NaN,"13,071 (end 2013)[1]",NaN,NaN,AGEAS HOUSE HAMPSHIRE CORPORATE PARK,TEMPLARS WAY,EASTLEIGH,HAMPSHIRE,NaN,SO53 3YA
1,Commercial and Personal,Assurant General Insurance Limited,ASSURANT GENERAL INSURANCE LIMITED,02341082,2341082,02341082,0.548802,1.060667,1.060667,$11.650 billion (2018)[2],...,NaN,12452 (2018)[6],NaN,NaN,EMERALD BUILDINGS,WESTMERE DRIVE,CREWE,CHESHIRE,UNITED KINGDOM,CW1 6UN
2,Commercial and Personal,AXA XL Catlin,XL CATLIN SERVICES SE,SE000103,SE000103,SE000103,0.266555,0.214050,0.214050,$10.546 billion (2016),...,NaN,"7,400",NaN,NaN,20 GRACECHURCH STREET,NaN,LONDON,NaN,NaN,EC3V 0BG
3,Commercial and Personal,CNA Hardy,CNA HARDY INTERNATIONAL SERVICES LIMITED,09849484,9849484,09849484,0.321837,0.379872,0.379872,NaN,...,NaN,NaN,NaN,NaN,20 FENCHURCH STREET,NaN,LONDON,NaN,ENGLAND,EC3M 3BY
4,Commercial and Personal,SOMPO HOLDINGS INC,INC HOLDINGS LIMITED,06560171,6560171,06560171,0.241599,0.139192,0.139192,"JP¥ 1,894,121 million (FY 2007)",...,NaN,"16,095 (2008)",NaN,NaN,NO 1 DUDLEY COURT NORTH,WATERFRONT EAST LEVEL STREET,BRIERLEY HILL,WEST MIDLANDS,NaN,DY5 1XP


In [5]:
def company_search(company):
    apiKey = "N673Z25KBNF3GK0ZIZP7hgOjvDc0LEP5g7xw1Pb5"
    url1 = "https://api.companieshouse.gov.uk/search/companies?q="
#     url2 = "&items_per_page=100&start_index="
    numbers = (0,100,200,300)
    
    data = []
#     for n in numbers:
    url_full = url1+company#+url2+str(n)
    r = requests.get(url_full, auth=(apiKey, '')).json()
    for i in range(0,r['total_results']):
        try:
            word = company
            name = r['items'][i]['title']
            company_no = r['items'][i]['company_number']
            company_type = r['items'][i]['company_type']
            link_self = r['items'][i]['links']['self']
            company_status = r['items'][i]['company_status']
            incorporation_date = r['items'][i]['date_of_creation']
            dissolution_date = r['items'][i]['description']
            address = r['items'][i]['address_snippet']
            if len(r['items'][i]['matches']['snippet']) == 0:
                previous_name = None
                ceased_on = None
                effective_from = None
            else:
                previous_name = requests.get("https://api.companieshouse.gov.uk/company/"+company_no, auth=(apiKey, '')).json()["previous_company_names"][0]['name']
                ceased_on = requests.get("https://api.companieshouse.gov.uk/company/"+company_no, auth=(apiKey, '')).json()["previous_company_names"][0]['ceased_on']
                effective_from = requests.get("https://api.companieshouse.gov.uk/company/"+company_no, auth=(apiKey, '')).json()["previous_company_names"][0]['effective_from']
        except (IndexError, KeyError):
            data.append(None)
        data.append((word, name,company_no,company_type,link_self,company_status,incorporation_date,dissolution_date,address, previous_name, ceased_on,effective_from))
    return data

In [131]:
data = company_search('AIG')

In [132]:
df = pd.DataFrame(data,columns=["search_query","name","company_no","company_type","link_self","status","incorporation_date",
                                         "dissolution_date","address","previous_name","ceased_on","effective_from"])

In [136]:
df.head()

,search_query,name,company_no,company_type,link_self,status,incorporation_date,dissolution_date,address,previous_name,ceased_on,effective_from
0,AIG,AIG LIMITED,03022299,ltd,/company/03022299,active,1995-02-15,03022299 - Incorporated on 15 February 1995,"8 Castle Square, Benson, Wallingford, Oxfords...",RINGROSE AIG LIMITED,1996-03-28,1995-02-15
1,AIG,AIG ACADEMY LTD,10962357,ltd,/company/10962357,dissolved,2017-09-14,10962357 - Dissolved on 19 February 2019,"68 Faircross Avenue, Barking, United Kingdom,...",None,None,None
2,AIG,AIG ACCOUNTING SOLUTIONS LIMITED,03893461,ltd,/company/03893461,active,1999-12-13,03893461 - Incorporated on 13 December 1999,"61 Lynn Road, Ely, Cambridgeshire, CB6 1DD",None,None,None
3,AIG,AIG + PARTNERS LTD,11824968,ltd,/company/11824968,active,2019-02-13,11824968 - Incorporated on 13 February 2019,"46 Syon Lane, Isleworth, Middlesex, England, T...",None,None,None
4,AIG,AIG ASSET MANAGEMENT (EUROPE) LIMITED,04665011,ltd,/company/04665011,active,2003-02-12,04665011 - Incorporated on 12 February 2003,"58 Fenchurch Street, London, EC3M 4AB",AIG GLOBAL INVESTMENT (EUROPE) LTD,2009-04-06,2003-02-13


In [12]:
def get_address(add):
    """
    Provides address string from dictionary of input
    """
    if add!=None:
        a = set(["address_line_1","address_line_2","care_of","country","locality","po_box","postal_code","premises","region"])
        b = set(add.keys())
        c = a-b
        d = dict.fromkeys(c, None)
        add.update(d)
        add

        lis = list((add['care_of'], add['address_line_1'], add['address_line_2'], add['premises'], add['locality'],
                    add['region'], add['po_box'], add['country'], add['postal_code']))
        lis = list(filter(None, lis))
        address = ', '.join(lis)
    else:
        address = None
    return address

In [13]:
def company_profile(companyID):
    apiKey = "N673Z25KBNF3GK0ZIZP7hgOjvDc0LEP5g7xw1Pb5"
    url1 = 'https://api.companieshouse.gov.uk/company/'
    url_full = url1+str(companyID).zfill(8)
    r = requests.get(url_full, auth=(apiKey, '')).json()
    try:
        # Non returned variables
        reg_address = r.get('registered_office_address')
        accounts = r.get('accounts')
        previous = r.get('previous_company_names')

        # Returned Variables
        ID = r.get('company_number')
        name = r.get('company_name')
        company_type = r.get('type')
        jurisdiction = r.get('jurisdiction')
        address = get_address(reg_address)
        incorporation_date = r.get('date_of_creation')
        if accounts != None:
            has_accounts_overdue = accounts.get('overdue')
        else:
            has_accounts_overdue = None
        has_insolvency_history = r.get('has_insolvency_history')
        liquidated = r.get('has_been_liquidated')
        can_file = r.get('can_file')
        
        # Support Variables
        try:
            previous_name = previous[0].get('name')
            ceased_on = previous[0].get('ceased_on')
            effective_from = previous[0].get('effective_from')
            
            previous_detail = previous_name+' ('+ effective_from +' to '+ ceased_on +')'
        except:
            previous_detail = None
            
    except (IndexError, KeyError):
        data.append(None)
    data = {
        'ID':ID,
        'name':name,
        'company_type':company_type,
        'jurisdiction':jurisdiction,
        'address':address,
        'incorporation_date':incorporation_date,
        'previous_detail':previous_detail,
        'has_accounts_overdue':has_accounts_overdue,
        'has_insolvency_history':has_insolvency_history,
        'liquidated':liquidated,
        'can_file':can_file
    }
    return data

In [14]:
# data = company_profile('00354568')
data_extracted = {
    'ID':[],
    'name':[],
    'company_type':[],
    'jurisdiction':[],
    'address':[],
    'incorporation_date':[],
    'previous_detail':[],
    'has_accounts_overdue':[],
    'has_insolvency_history':[],
    'liquidated':[],
    'can_file':[]
}

for company in tqdm(df['Company Number.1']):
    data = company_profile(str(company))
    for key in list(data.keys()):
        data_extracted[key].append(data[key])
    time.sleep(0.75)

In [32]:
pd.DataFrame(data_extracted)

,ID,name,company_type,jurisdiction,address,incorporation_date,previous_detail,has_accounts_overdue,has_insolvency_history,liquidated,can_file
0,01227004,ADVENT UNDERWRITING LIMITED,ltd,england-wales,"2nd Floor, 2 Minster Court, London, EC3R 7BB",1975-09-19,B.F. CAUDLE AGENCIES LIMITED (1975-09-19 to 20...,False,False,False,True
1,03413859,AEGIS MANAGING AGENCY LIMITED,ltd,england-wales,"33 Gracechurch Street, London, EC3V 0BT",1997-07-29,None,False,False,False,True
2,07249776,ALLIED WORLD MANAGING AGENCY LIMITED,ltd,england-wales,"19th Floor 20 Fenchurch Street, London, Englan...",2010-05-11,2232 SERVICES LIMITED (2010-05-11 to 2014-03-11),False,False,False,True
3,04434499,AMTRUST SYNDICATES LIMITED,ltd,england-wales,"Exchequer Court, 33 St Mary Axe, London, Engla...",2002-05-09,ANV SYNDICATES LIMITED (2013-12-06 to 2016-11-10),False,False,False,True
4,06646629,ANTARES MANAGING AGENCY LIMITED,ltd,england-wales,"21 Lime Street, London, England, EC3M 7HB",2008-07-15,MINMAR (910) LIMITED (2008-07-15 to 2008-09-04),False,False,False,True
5,09181578,APOLLO SYNDICATE MANAGEMENT LIMITED,ltd,england-wales,"One Bishopsgate, London, EC2N 3AQ",2014-08-19,None,False,False,False,True
6,06645822,ARCH UNDERWRITING AT LLOYD'S LTD,ltd,england-wales,"5th Floor, Plantation Place South, 60 Great To...",2008-07-14,ARCH UNDERWRITING AT LLOYD'S LIMITED (2008-08-...,False,False,False,True
7,03632880,ARGENTA SYNDICATE MANAGEMENT LIMITED,ltd,england-wales,"5th Floor 70 Gracechurch Street, London, Engla...",1998-09-11,SACKVILLE SYNDICATE MANAGEMENT LIMITED (1999-0...,False,False,False,True
8,03768610,ARGO MANAGING AGENCY LIMITED,ltd,england-wales,"1 Fen Court, London, United Kingdom, EC3M 5BN",1999-05-12,HERITAGE MANAGING AGENCY LIMITED (1999-07-02 t...,False,False,False,True
9,05887810,ARK SYNDICATE MANAGEMENT LIMITED,ltd,england-wales,"30 Fenchurch Avenue, London, EC3M 5AD",2006-07-26,DARNER NYMPH LIMITED (2006-07-26 to 2007-02-01),False,False,False,True


In [15]:
df1 = pd.DataFrame(data_extracted)

In [16]:
df1.to_excel('Companieshouse_Data.xlsx',sheet_name = 'all_sector', index = False, engine='xlsxwriter')